In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy import stats
import early_warning_detection_functions
import matplotlib.pyplot as plt


# Read data

In [2]:
df_aps = pd.read_parquet('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Results/data_manuscript_warning_phc_only_imed.parquet')

df_otc = pd.read_parquet('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Results/data_manuscript_warning_otc_only_imed.parquet')

df_aih = pd.read_parquet('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Results/data_manuscript_warning_aih_imed2.parquet')



# Process data

In [11]:
dta1 = df_aps[['nm_uf', 'co_imed', 'Nome_imed', 'year_week', 'year_week_ts',
       'epidemi_cal_start', 'epidemi_cal_end', 'atend_ivas','atend_ivas_4',
      'fitted_values_phc_only', 'residuals_phc_only']]

In [12]:
dta2 = df_otc[['co_imed', 'year_week', 'num_otc_ivas', 'num_otc_ivas_4', 'fitted_values_otc_only', 'residuals_otc_only']]

In [13]:
dta = dta1.merge(dta2, on = ['co_imed', 'year_week'], how = 'left')

In [17]:
dta.head()

,nm_uf,co_imed,Nome_imed,year_week,year_week_ts,epidemi_cal_start,epidemi_cal_end,atend_ivas,atend_ivas_4,fitted_values_phc_only,residuals_phc_only,num_otc_ivas,num_otc_ivas_4,fitted_values_otc_only,residuals_otc_only
0,Acre,120003,Sena Madureira,2022-47,2022-11-21,2022-11-20,2022-11-26,347,0.00,87.322732,-6.746032,1191.0,0.00,516.987048,-3.535520
1,Acre,120003,Sena Madureira,2022-48,2022-11-28,2022-11-27,2022-12-03,292,0.00,87.322732,-6.746032,1202.0,0.00,517.716184,-3.535918
2,Acre,120003,Sena Madureira,2022-49,2022-12-05,2022-12-04,2022-12-10,291,0.00,87.322732,-6.746032,1813.0,0.00,518.446348,-3.536315
3,Acre,120003,Sena Madureira,2022-50,2022-12-12,2022-12-11,2022-12-17,454,346.00,87.322732,5.419730,1317.0,1380.75,519.177542,1.166502
4,Acre,120003,Sena Madureira,2022-51,2022-12-19,2022-12-18,2022-12-24,394,357.75,351.261116,0.057257,1189.0,1380.25,1203.450129,0.140221


In [18]:
set_region = dta[dta.co_imed == 120003]

In [20]:
set_region.columns

Index(['nm_uf', 'co_imed', 'Nome_imed', 'year_week', 'year_week_ts',
       'epidemi_cal_start', 'epidemi_cal_end', 'atend_ivas', 'atend_ivas_4',
       'fitted_values_phc_only', 'residuals_phc_only', 'num_otc_ivas',
       'num_otc_ivas_4', 'fitted_values_otc_only', 'residuals_otc_only'],
      dtype='object')

In [ ]:
# Control limits control limits for different levels 

lst = []

for w in [0,0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]:
    
    for code in dta.co_imed.unique():
        
        set_region = dta[dta.co_imed == code]

        sigma_t_phc_only = set_region.fitted_values_phc_only.rolling(window=5).std()
    
        UCL_phc_only = set_region.fitted_values_phc_only + w * sigma_t_phc_only
        
        set_region = set_region.assign(warning_phc_w = (set_region['atend_ivas_4'] >= set_region.UCL_phc_only).astype(int))
        
         lst.append(set_region)